In [1]:
from KGS import * # Utilizing my own package for data analysis. Remember to also send enviroment when sharing. 
import scipp as sc
from IPython.display import Image
from TubeMeasurment_mccode import BIFROST_measurement
mpl.rcParams['font.size']        = 20

In [2]:
#BIFROST_measurement.load_scan('Phonon_Pb_v2_','Pb_phonon_Data_1t_Cluster_corrected',set_Elim=(-0.5,7), Correct=True)

## Load the information

In [3]:
settings = np.loadtxt('Pb_phonon_Data_1t_Cluster_corrected').T

FileNotFoundError: Pb_phonon_Data_1t_Cluster_corrected not found.

## Organize into scipp array

In [ ]:

qx = sc.array(dims=["row"], values=settings[3], unit="Å^(-1)")
qy = sc.array(dims=["row"], values=settings[4], unit="Å^(-1)")
deltaE = sc.array(dims=["row"], values=settings[2], unit="meV")
Q = sc.array(dims=["row"], values=np.sqrt(settings[3]**2+settings[4]**2), unit="Å^(-1)")

recording = sc.DataArray(
    data=sc.array(dims=["row"], values=settings[0], unit="arb. units", variances=settings[1]**2),  #, variances=settings[1]
    coords={"qx": qx, "qy": qy, "deltaE":deltaE, "Q":Q}
)
recording

In [ ]:
sc.show(recording)

In [ ]:
qx_bins = round((max(recording.coords['qx'].values) - min(recording.coords['qx'].values))/0.02)
qy_bins = round((max(recording.coords['qy'].values) - min(recording.coords['qy'].values))/0.02)
E_bins = round((max(recording.coords['deltaE'].values) - min(recording.coords['deltaE'].values))/0.1)

print(qx_bins, qy_bins, E_bins)
print(max(recording.coords['qx'].values))

## Perform the rebinning

In [ ]:
import scipp as sc


da = recording.bin(qx=qx_bins, qy=qy_bins, deltaE=E_bins)
sliced = da.bins.mean()
#sliced.values[(da.bins.size() == sc.index(0)).values] = np.nan
sliced.save_hdf5(filename='Pb_phonon_rebinned.hdf5')

sc.show(da)

In [ ]:
#np.savetxt('Phonon_Pb_rebinned_1', matrix, delimiter='\t', fmt='%f') # pip install pythreejs

In [ ]:
#slicedQ = recording.bin(deltaE=150, Q=200)

In [ ]:
#slicedQ = slicedQ.bins.mean()
#slicedQ

In [ ]:
E_min = -2 * sc.Unit('meV')
E_max = 10 * sc.Unit('meV')

Q_min = 0 * sc.Unit('Å^(-1)')
Q_max = 4.5 * sc.Unit('Å^(-1)')

#fig, ax = plt.subplots(1, 1, figsize=(6, 4))

#p5 = slicedQ['Q',Q_min:Q_max]['deltaE', E_min:E_max].plot(ax=ax, cmap='jet')

#ax.set_ylabel(r'$\Delta E$ [meV]')
#ax.set_xlabel(r'$|Q|$ [Å$^{-1}$]')

In [ ]:
#E_layer = 2 * sc.Unit('meV')

import matplotlib.colors as colors

qx_min = 0 * sc.Unit('Å^(-1)')
qx_max = 2 * sc.Unit('Å^(-1)')

qy_min = -2 * sc.Unit('Å^(-1)')
qy_max = 4.0 * sc.Unit('Å^(-1)')


import ipywidgets as ipw
import plopp.widgets as pw
import plopp as pp
import mpld3



def update_plot(amp):
    E_layer = amp * sc.Unit('meV')
    plt.figure(figsize=(6,4))
    p1 = sliced['deltaE', E_layer]['qy', qy_min:qy_max].transpose()#.plot(cmap='jet')
    im = plt.pcolormesh(p1.coords['qx'].values, p1.coords['qy'].values, p1.values, cmap='jet',  vmin=0, vmax=0.0001) # vmin=0, vmax=3e-5
    cbar = plt.colorbar(im)
    cbar.set_label('I [a.u.]')
    plt.xlim(-2,2)
    plt.ylim(0,4)
    plt.xlabel('$q_x$ [1/Å]')
    plt.ylabel('$q_y$ [1/Å]')
    plt.title(f"$\Delta E =$ {amp:.2f} meV")
    plt.savefig('Pb_phonon_BIFROST_qxqy_mid.pdf', format='pdf', bbox_inches='tight')
    

w = ipw.FloatSlider( 
    value=3.44,
    min=min(sliced.coords['deltaE'].values),
    max=7.0,
    step=(sliced.coords['deltaE'].values[1]-sliced.coords['deltaE'].values[0]),
    description="Delta_E:",
    disabled=False,
    continuous_update=True,
    orientation="horizontal",
    readout=True,
    readout_format=".2f",
)
# Display the slider

ipw.interactive(update_plot, amp=w)



In [ ]:
# da = recording.bin(qx=120, qy=120, deltaE=40)
# sliced = da.bins.sum()
# sliced.values[(da.bins.size() == sc.index(0)).values] = np.nan

qxE = recording.bin(qx=qx_bins, qy=qy_bins, deltaE=E_bins)
sliced_qxE = qxE.bins.mean()
#sliced_qxE.values[(qxE.bins.size() == sc.index(0)).values] = np.nan

# Sliced along $q_x$

In [ ]:

qx_min = -1 * sc.Unit('Å^(-1)')
qx_max = 2 * sc.Unit('Å^(-1)')

E_min = -0.05 * sc.Unit('meV')
E_max = 7.0 * sc.Unit('meV')


def update_plot_qxE(amp):
    qy_layer = amp * sc.Unit('Å^(-1)')
    plt.figure(figsize=(6,4))
    p1 = sliced_qxE['qy', qy_layer]['deltaE', E_min:E_max].transpose()#.plot(cmap='jet')
    im = plt.pcolormesh(p1.coords['qx'].values, p1.coords['deltaE'].values, p1.values, cmap='jet', vmin=0, vmax=0.0001) # norm=colors.LogNorm()
    cbar = plt.colorbar(im)
    cbar.set_label('I [a.u.]')
    plt.xlim(-2,2.5)
    plt.ylim(0,7)
    plt.xlabel('$q_x$ [1/Å]')
    plt.ylabel('$\Delta E$ [meV]')
    plt.title(f"$q_y =$ {amp:.2f} Å$^{-1}$")
    plt.savefig('Pb_phonon_BIFROST_qxE_high.pdf', format='pdf', bbox_inches='tight')
    

w_qxE = ipw.FloatSlider( 
    value=2.73,
    min=min(sliced_qxE.coords['qy'].values),
    max=4,
    step=(sliced_qxE.coords['qy'].values[1]-sliced_qxE.coords['qy'].values[0]),
    description="q_y:",
    disabled=False,
    continuous_update=True,
    orientation="horizontal",
    readout=True,
    readout_format=".2f",
)
# Display the slider



ipw.interactive(update_plot_qxE, amp=w_qxE)


## Cut integrating energy - x-axis $q_x$

In [ ]:
mpl.rcParams['font.size']        = 15

c1_qy_min = 2.53 * sc.Unit('Å^(-1)')

c1_Emin = 1.8 * sc.Unit('meV')
c1_Emax = 2.2 * sc.Unit('meV')

c1_qxmin = -0.5 * sc.Unit('Å^(-1)')
c1_qxmax = 0.5 * sc.Unit('Å^(-1)')


c1 = sliced_qxE['qy', c1_qy_min]['qx',c1_qxmin:c1_qxmax]['deltaE', c1_Emin:c1_Emax]
c1 = sc.sum(c1, dim='deltaE')

qx_c1 = c1.coords['qx'].values
qx_c1 = (qx_c1[:-1] + qx_c1[1:]) / 2.0

E_c1_guess = {'A1':0.0002, 'mu1':-0.2, 'sigma1': 0.01,
                 'A2': 0.0002, 'mu2':0.2, 'sigma2': 0.01,}
E_c1_limit = {'A2': (0.0001,0.0003), 'mu2':(0.1,0.3)}

Ecut_fit1 = fit(qx_c1, c1.values, np.sqrt(c1.variances), gaussN, E_c1_guess, limits=E_c1_limit)

Ecut_fit1.hesse()


## Cuts integrating qx - x-axis energy

In [ ]:
c2_qy_min = 2.53 * sc.Unit('Å^(-1)')

c2_Emin = 1.0 * sc.Unit('meV')
c2_Emax = 3.0 * sc.Unit('meV')

c2_qxmin = 0.2 * sc.Unit('Å^(-1)')
#c2_qxmax = 0.23 * sc.Unit('Å^(-1)')


c2 = sliced_qxE['qy', c2_qy_min]['qx',c2_qxmin]['deltaE', c2_Emin:c2_Emax]
#c2 = sc.sum(c2, dim='qx')

qx_c2 = c2.coords['deltaE'].values
qx_c2 = (qx_c2[:-1] + qx_c2[1:]) / 2.0

c2
E_c2_guess = {'A1':0.00005, 'mu1':0.2, 'sigma1': 1,}
E_c2_limit = {'A2': (0.0001,0.0003), 'mu2':(0.1,0.3)}

Ecut_fit2 = fit(qx_c2, c2.values, np.sqrt(c2.variances), gaussN, E_c2_guess)

Ecut_fit2.hesse()

In [ ]:
c3_qy_min = 2.53 * sc.Unit('Å^(-1)')

c3_Emin = 1.0 * sc.Unit('meV')
c3_Emax = 3.0 * sc.Unit('meV')

c3_qxmin = -0.2 * sc.Unit('Å^(-1)')
#c3_qxmax = -0.18 * sc.Unit('Å^(-1)')


c3 = sliced_qxE['qy', c3_qy_min]['qx',c3_qxmin]['deltaE', c3_Emin:c3_Emax]
#c3 = sc.sum(c3, dim='qx')

qx_c3 = c3.coords['deltaE'].values
qx_c3 = (qx_c3[:-1] + qx_c3[1:]) / 2.0
c3
E_c3_guess = {'A1':0.00005, 'mu1':2, 'sigma1': 0.2,}
E_c3_limit = {'A1': (0.00001,0.0003), 'mu1':(1,3)}

Ecut_fit3 = fit(qx_c3, c3.values, np.sqrt(c3.variances), gaussN, E_c3_guess, limits=E_c3_limit)

Ecut_fit3.hesse()
c3

## Plot 

In [ ]:
qx_range = np.linspace(min(qx_c1), max(qx_c1), 1000)
E_range = np.linspace(min(qx_c2), max(qx_c2), 1000)

fig, ax = plt.subplots(1,3, figsize=(13,3), sharey=True)
fig.subplots_adjust(wspace=0.05)

ax[0].errorbar(qx_c3, c3.values, np.sqrt(c3.variances), fmt='.', label='Data', color='#7570b3')
ax[0].plot(E_range, gaussN(E_range, *Ecut_fit3.values), color='#7570b3')
ax[0].set_title(r'$q_y$: [2.53:2.55] - $qx$: [-0.21 : -0.19]')
ax[0].set_xlabel(r'$\Delta E$ [meV]')
ax[0].set_ylabel('I [a.u.]')
ax[0].grid()

ax[1].errorbar(qx_c1, c1.values, np.sqrt(c1.variances), fmt='.', label='Data', color='#1b9e77')
ax[1].plot(qx_range, gaussN(qx_range, *Ecut_fit1.values), color='#1b9e77')
ax[1].set_title(r'$q_y$: [2.53:2.55] - $\Delta E$: [1.9 : 2.1]')
ax[1].set_xlabel(r'$q_x$ [Å$^{-1}$]')
ax[1].grid()
#
ax[2].errorbar(qx_c2, c2.values, np.sqrt(c2.variances), fmt='.', label='Data', color='#d95f02')
ax[2].plot(E_range, gaussN(E_range, *Ecut_fit2.values), color='#d95f02')
ax[2].set_title(r'$q_y$: [2.53:2.55] - $q_x$: [0.19 : 0.21]')
ax[2].set_xlabel(r'$\Delta E$ [meV]')
ax[2].grid()
plt.savefig('qx_cuts.pdf', format='pdf', bbox_inches='tight')


# Sliced along $q_y$

In [ ]:

qy_min = -1 * sc.Unit('Å^(-1)')
qy_max = 2 * sc.Unit('Å^(-1)')

E_min = -0.05 * sc.Unit('meV')
E_max = 7.0 * sc.Unit('meV')


def update_plot_qyE(amp):
    qx_layer = amp * sc.Unit('Å^(-1)')
    plt.figure(figsize=(6,4))
    p1 = sliced_qxE['qx', qx_layer]['deltaE', E_min:E_max].transpose()#.plot(cmap='jet')
    im = plt.pcolormesh(p1.coords['qy'].values, p1.coords['deltaE'].values, p1.values, cmap='jet', vmin=0, vmax=0.0001)
    cbar = plt.colorbar(im)
    cbar.set_label('I [a.u.]')
    plt.xlim(1.5,4)
    plt.ylim(0,7)
    plt.xlabel('$q_y$ [1/Å]')
    plt.ylabel('$\Delta E$ [meV]')
    plt.title(f"$q_x =$ {amp:.2f} Å$^{-1}$")
    plt.savefig('Pb_phonon_BIFROST_qyE_low.pdf', format='pdf', bbox_inches='tight')
    

w_qyE = ipw.FloatSlider( 
    value=0.17,
    min=min(sliced_qxE.coords['qx'].values),
    max=4,
    step=(sliced_qxE.coords['qx'].values[1]-sliced_qxE.coords['qx'].values[0]),
    description="q_x:",
    disabled=False,
    continuous_update=True,
    orientation="horizontal",
    readout=True,
    readout_format=".2f",
)
# Display the slider


ipw.interactive(update_plot_qyE, amp=w_qyE)

## Cuts integrating $q_y$ 

In [ ]:
c4_qx= 0 * sc.Unit('Å^(-1)')

c4_Emin = 1.8 * sc.Unit('meV')
c4_Emax = 2.2 * sc.Unit('meV')

c4_qymin = 2 * sc.Unit('Å^(-1)')
c4_qymax = 3 * sc.Unit('Å^(-1)')


c4 = sliced_qxE['qx', c4_qx]['qy',c4_qymin:c4_qymax]['deltaE', c4_Emin:c4_Emax]
c4 = sc.sum(c4, dim='deltaE')

qy_c4 = c4.coords['qy'].values
qy_c4 = (qy_c4[:-1] + qy_c4[1:]) / 2.0

E_c4_guess = {'A1':0.0002, 'mu1':2.33, 'sigma1': 0.01,
                 'A2': 0.0002, 'mu2':2.73, 'sigma2': 0.01,}
E_c4_limit = {'A2': (0.0001,0.0004)}

Ecut_fit4 = fit(qy_c4, c4.values, np.sqrt(c4.variances), gaussN, E_c4_guess, limits=E_c4_limit)

Ecut_fit4.hesse()

In [ ]:
qx_layer = 0 * sc.Unit('Å^(-1)')

c5_qymin = 2.33 * sc.Unit('Å^(-1)')
#c5_qymax = 2.36 * sc.Unit('Å^(-1)')

c5_Emin = 1 * sc.Unit('meV')
c5_Emax = 3 * sc.Unit('meV')

c5 = sliced_qxE['qx', qx_layer]['qy',c5_qymin]['deltaE', c5_Emin:c5_Emax]
#c5 = sc.sum(c5, dim='qy')

qy_c5 = c5.coords['deltaE'].values
qy_c5 = (qy_c5[:-1] + qy_c5[1:]) / 2.0

E_c5_guess = {'A1':0.00005, 'mu1':2, 'sigma1': 0.2,}
E_c5_limit = {'A1': (0.00001,0.0003), 'mu1':(1,3)}

Ecut_fit5 = fit(qy_c5, c5.values, np.sqrt(c5.variances), gaussN, E_c5_guess, limits=E_c5_limit)

Ecut_fit5.hesse()


In [ ]:
c6_qymin = 2.73 * sc.Unit('Å^(-1)')
#c6_qymax = 2.76 * sc.Unit('Å^(-1)')

c6_Emin = 1 * sc.Unit('meV')
c6_Emax = 3 * sc.Unit('meV')

c6 = sliced_qxE['qx', qx_layer]['qy',c6_qymin]['deltaE', c6_Emin:c6_Emax]
#c6 = sc.sum(c6, dim='qy')

qy_c6 = c6.coords['deltaE'].values
qy_c6 = (qy_c6[:-1] + qy_c6[1:]) / 2.0

E_c6_guess = {'A1':0.00005, 'mu1':2, 'sigma1': 0.2,}
E_c6_limit = {'A1': (0.00001,0.0003), 'mu1':(1,3)}

Ecut_fit6 = fit(qy_c6, c6.values, np.sqrt(c6.variances), gaussN, E_c6_guess, limits=E_c6_limit)

Ecut_fit6.hesse()

c5


In [ ]:
q_range2 = np.linspace(min(qy_c4), max(qy_c4),1000)
E_range2 = np.linspace(min(qy_c6), max(qy_c6),1000)

fig, ax = plt.subplots(1,3, figsize=(13,3), sharey=True)
fig.subplots_adjust(wspace=0.05)

ax[0].errorbar(qy_c5, c5.values, np.sqrt(c5.variances), fmt='.', label='Data', color='#e7298a')
ax[0].plot(E_range2, gaussN(E_range2, *Ecut_fit5.values), color='#e7298a')
ax[0].set_title(r'$q_x$: [-0.01 : 0.01] - $q_y$: [2.33 :2.35]')
ax[0].set_xlabel(r'$\Delta E$ [meV]')
ax[0].set_ylabel('I [a.u.]')
ax[0].grid()

ax[1].errorbar(qy_c4, c4.values, np.sqrt(c4.variances), fmt='.', label='Data', color='red')
ax[1].plot(q_range2, gaussN(q_range2, *Ecut_fit4.values), color='red')
ax[1].set_title(r'$q_x$: [-0.01 : 0.01] - $\Delta E$: [1.8 : 2.2]')
ax[1].set_xlabel(r'$q_y$ [Å$^{-1}$]')
ax[1].grid()

ax[2].errorbar(qy_c6, c6.values, np.sqrt(c6.variances), fmt='.', label='Data', color='cyan')
ax[2].plot(E_range2, gaussN(E_range2, *Ecut_fit6.values), color='cyan')
ax[2].set_title(r'$q_x$: [-0.01 : 0.01] - $q_y$: [2.72 : 2.74]')
ax[2].set_xlabel(r'$\Delta E$ [meV]')
ax[2].grid()

plt.savefig('qy_cuts.pdf', format='pdf', bbox_inches='tight')

In [ ]:
from matplotlib.patches import Polygon

E_layer = 2.0 * sc.Unit('meV')
fig, ax = plt.subplots(1,1,figsize=(6,4))
p1 = sliced['deltaE', E_layer]['qy', qy_min:qy_max].transpose()#.plot(cmap='jet')
im = ax.pcolormesh(p1.coords['qx'].values, p1.coords['qy'].values, p1.values, cmap='jet',  vmin=0, vmax=0.0001) # vmin=0, vmax=3e-5
cbar = plt.colorbar(im)
cbar.set_label('I [a.u.]')
ax.set_xlim(-0.3,0.3)
ax.set_ylim(2.2,2.8)
ax.set_xlabel('$q_x$ [1/Å]')
ax.set_ylabel('$q_y$ [1/Å]')
ax.set_title(f"$\Delta E =$ 2.0 meV")
vertices2 = [(-0.01, 2.33), (0.01, 2.33), (0.01, 2.35),(-0.01, 2.35)]
parallelogram2 = Polygon(vertices2, closed=True, edgecolor='#e7298a', facecolor='none')
ax.add_patch(parallelogram2)

vertices3 = [(-0.01, 2.73), (0.01, 2.73), (0.01, 2.75),(-0.01, 2.75)]
parallelogram3 = Polygon(vertices3, closed=True, edgecolor='cyan', facecolor='none')
ax.add_patch(parallelogram3)

vertices4 = [(-0.21, 2.53), (-0.19, 2.53), (-0.19, 2.55),(-0.21, 2.55)]
parallelogram4 = Polygon(vertices4, closed=True, edgecolor='#7570b3', facecolor='none')
ax.add_patch(parallelogram4)

vertices5 = [(0.21, 2.53), (0.19, 2.53), (0.19, 2.55),(0.21, 2.55)]
parallelogram5 = Polygon(vertices5, closed=True, edgecolor='#d95f02', facecolor='none')
ax.add_patch(parallelogram5)

plt.savefig('Pb_phonon_BIFROST_qxqy_low.pdf', format='pdf', bbox_inches='tight')

In [ ]:
qy_layer = 2.53 * sc.Unit('Å^(-1)')
fig, ax = plt.subplots(1,1,figsize=(6,4))
p1 = sliced_qxE['qy', qy_layer]['deltaE', E_min:E_max].transpose()#.plot(cmap='jet')
im = ax.pcolormesh(p1.coords['qx'].values, p1.coords['deltaE'].values, p1.values, cmap='jet', vmin=0, vmax=0.0001) # norm=colors.LogNorm()
cbar = plt.colorbar(im)
cbar.set_label('I [a.u.]')
ax.set_xlim(-2,2.5)
ax.set_ylim(0,7)
ax.set_xlabel('$q_x$ [1/Å]')
ax.set_ylabel('$\Delta E$ [meV]')
ax.set_title(f"$q_y =$ 2.53 Å$^{-1}$")

vertices5 = [(-0.21, 1), (-0.19, 1), (-0.19, 3),(-0.21, 3)]
parallelogram5 = Polygon(vertices5, closed=True, edgecolor='#7570b3', facecolor='none')
ax.add_patch(parallelogram5)

vertices6 = [(0.21, 1), (0.19, 1), (0.19, 3),(0.21, 3)]
parallelogram6 = Polygon(vertices6, closed=True, edgecolor='#d95f02', facecolor='none')
ax.add_patch(parallelogram6)

vertices6 = [(0.5, 1.8), (-0.5, 1.8), (-0.5, 2.2),(0.5, 2.2)]
parallelogram6 = Polygon(vertices6, closed=True, edgecolor='#1b9e77', facecolor='none')
ax.add_patch(parallelogram6)

plt.savefig('Pb_phonon_BIFROST_qxE_mid.pdf', format='pdf', bbox_inches='tight')

In [ ]:
qx_layer = 0 * sc.Unit('Å^(-1)')
fig, ax = plt.subplots(1,1,figsize=(6,4))
p1 = sliced_qxE['qx', qx_layer]['deltaE', E_min:E_max].transpose()#.plot(cmap='jet')
im = ax.pcolormesh(p1.coords['qy'].values, p1.coords['deltaE'].values, p1.values, cmap='jet', vmin=0, vmax=0.0001)
cbar = plt.colorbar(im)
cbar.set_label('I [a.u.]')
ax.set_xlim(1.5,4)
ax.set_ylim(0,7)
ax.set_xlabel('$q_y$ [1/Å]')
ax.set_ylabel('$\Delta E$ [meV]')
ax.set_title(f"$q_x =$ 0 Å$^{-1}$")

vertices5 = [(2.33, 1), (2.35, 1), (2.35, 3),(2.33, 3)]
parallelogram5 = Polygon(vertices5, closed=True, edgecolor='#e7298a', facecolor='none')
ax.add_patch(parallelogram5)

vertices6 = [(2.72, 1), (2.74, 1), (2.74, 3),(2.72, 3)]
parallelogram6 = Polygon(vertices6, closed=True, edgecolor='cyan', facecolor='none')
ax.add_patch(parallelogram6)

vertices6 = [(3, 1.8), (2, 1.8), (2, 2.2),(3, 2.2)]
parallelogram6 = Polygon(vertices6, closed=True, edgecolor='red', facecolor='none')
ax.add_patch(parallelogram6)

plt.savefig('Pb_phonon_BIFROST_qyE_mid.pdf', format='pdf', bbox_inches='tight')

# Fitting phonon in (h,k)

In [ ]:
def to_h(qx):
    return qx / (2 * np.pi)

def to_k(qy):
    return qy / (2 * np.pi)

graph = {'h': to_h}
graph2 = {'k': to_k}

transformed = sliced.transform_coords(['h'], graph=graph)
transformed2 = transformed.transform_coords(['k'], graph=graph2)

## Making slices

In [ ]:
p1 = sliced_qxE['qy', qy_layer]['deltaE', E_min:E_max].transpose()
p1